In [1]:
from pathlib import Path

DATA_DIR = Path.cwd().parent / "data/train"


In [2]:
import numpy as np
import jax
import jax.numpy as jnp
import tensorflow as tf
from deeprte.dataset import (
    Split,
    load,
    get_nest_dict_shape,
    preprocess_grid,
    _load_and_split_dataset,
    _shard
)


In [8]:
from deeprte.utils import flat_dict_to_rte_data

dataset = np.load(DATA_DIR / "square_full_1.npz")

dataset = flat_dict_to_rte_data(dataset)

print(tf.nest.map_structure(lambda x: x.shape, dict(dataset)))


{'data': {'sigma_t': (2000, 40, 40), 'sigma_a': (2000, 40, 40), 'psi_label': (2000, 40, 40, 24), 'phi': (2000, 40, 40), 'psi_bc': (2000, 160, 12)}, 'grid': {'r': (40, 40, 2), 'v': (24, 2), 'w_angle': (24,), 'rv_prime': (160, 12, 4), 'w_prime': (160, 12)}}


In [13]:
dataset["data"]["psi_bc"].dtype

dtype('float32')

In [14]:
data["w_angle"].dtype

dtype('float32')

In [5]:
start, end = _shard(Split.TEST, jax.process_index(), jax.process_count())
(ds, grid), split, indices2 = _load_and_split_dataset(DATA_DIR / "square_full_1.npz", Split.TEST, end=end, from_=start)

In [4]:
import matplotlib.pyplot as plt

data, grid = dataset["data"], dataset["grid"]

idx = np.random.randint(500)

# pred_rho = pred_rho.reshape(-1, 40, 40).squeeze()
phi = data["phi"]
r = grid["r"]


# fig, _axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))
# fig.subplots_adjust(hspace=0.3)
# axs = _axs.flatten()

fig = px.density_contour(phi[idx])
# fig.show()

# cs = axs[0].contourf(
#     r[..., 0],
#     r[..., 1],
#     phi[idx],
# )
# axs[0].clabel(cs, inline=True, fontsize=10)
# fig.colorbar(cs, ax=axs[0])

# axs[1].plot(
#     phi[idx, 0],
# )

# plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import time

i = np.random.randint(500)


data, grid = flat_dict_to_rte_data(dataset)

data_i = tf.nest.map_structure(lambda x: x[i : i + 1].reshape(1, -1), data)

psi_bc = data_i["psi_bc"]
sigma = np.stack([data_i["sigma_t"], data_i["sigma_a"]], axis=-1)
r, v, rv_prime, w_prime, w_angle = (
    grid["r"],
    grid["v"],
    grid["rv_prime"],
    grid["w_prime"],
    grid["w_angle"],
)

t_prev = time.time()
pred_rho = sol.rho(
    params, None, r, F(r, sigma), F(rv_prime, w_prime * psi_bc), (v, w_angle)
)
pred_rho.block_until_ready()
dt = time.time() - t_prev

print(dt)


In [40]:
ds = load(
    DATA_DIR / "rte_2d_converted.npz",
    Split.VALID,
    is_training=False,
    batch_sizes=[jax.local_device_count(), 3],
    collocation_sizes=None,
    repeat=None,
)

# init_inputs = load_dummy_data(
#     DATA_DIR / "rte_2d_converted.npz", Split.TRAIN_AND_VALID, device_count=5
# )

# get_nest_dict_shape(init_inputs)


In [41]:
for i, d in enumerate(ds):
    if i <= 5:
        print(get_nest_dict_shape(d))
    else:
        break


INFO:absl:Data shapes, sigma_t: (500, 40, 40), sigma_a: (500, 40, 40), psi_bc: (500, 40, 12), psi_label: (500, 40, 40, 24), phi: (500, 40, 40)
INFO:absl:Grid shapes, w_angle: (24,), v: (24, 2), r: (40, 40, 2), rv_prime: (40, 12, 4), w_prime: (40, 12)


{'inputs': ((8, 38400, 2), (8, 38400, 2), F(x=(8, 1600, 2), y=(8, 3, 1600, 2)), F(x=(8, 480, 4), y=(8, 3, 480))), 'labels': (8, 3, 38400)}
{'inputs': ((8, 38400, 2), (8, 38400, 2), F(x=(8, 1600, 2), y=(8, 3, 1600, 2)), F(x=(8, 480, 4), y=(8, 3, 480))), 'labels': (8, 3, 38400)}
{'inputs': ((8, 38400, 2), (8, 38400, 2), F(x=(8, 1600, 2), y=(8, 3, 1600, 2)), F(x=(8, 480, 4), y=(8, 3, 480))), 'labels': (8, 3, 38400)}
{'inputs': ((8, 38400, 2), (8, 38400, 2), F(x=(8, 1600, 2), y=(8, 3, 1600, 2)), F(x=(8, 480, 4), y=(8, 3, 480))), 'labels': (8, 3, 38400)}


In [ ]:
from deeprte.dataset import get_nest_dict_shape

for d in ds.take(1):
    print(get_nest_dict_shape(d))
    r, v = d["interior"][0], d["interior"][1]
    print(np.sum((r[1] - r[0]) ** 2))
    print(np.sum((v[1] - v[0]) ** 2))
